import numpy as np
import os
from helpers import *
from PIL import ImageFile

import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array, array_to_img
from sklearn.model_selection import train_test_split


### Load Training Images and Grountruth versions

In [2]:
# Load the training set
root_dir = "./training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
print("Loading " + str(n) + " images")
imgs = [np.asarray(load_image((image_dir + files[i]))) for i in range(n)]
imgs = np.array(imgs)
gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [np.asarray(load_image((gt_dir + files[i]))) for i in range(n)]
gt_imgs = np.array(gt_imgs)
print(imgs.shape)
print(gt_imgs.shape)

Loading 100 images
Loading 100 images
(100, 400, 400, 3)
(100, 400, 400)


### Train and Validation Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(imgs, gt_imgs, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(80, 400, 400, 3)
(20, 400, 400, 3)
(80, 400, 400)
(20, 400, 400)


### Run Logistic Regression

In [4]:
from logistic_model import LogisticModel
lr_model = LogisticModel()

In [5]:
lr_model.initialize()
lr_model.train(y_train, X_train)

Training completed


C:\Users\neera\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [6]:
Z = lr_model.classify(X_test)
    
# Calculate ground-truth labels
img_patches_gt = create_patches_gt(y_test, 16, 16)
y_real = np.mean(img_patches_gt, axis=(1, 2)) > 0.25
print(get_classification_results(y_real, Z)  )



0.76032


### Run Naive CNN Model

In [7]:
from naive_cnn_model import NaiveCnnModel
cnn_model = NaiveCnnModel()

In [8]:
cnn_model.initialize()
cnn_model.train(y_train, X_train)

Initialize cnn
Epoch 1/10
50000/50000 [==============================] - 21s 420us/step - loss: 0.5382 - accuracy: 0.7478
Epoch 2/10
50000/50000 [==============================] - 18s 368us/step - loss: 0.5014 - accuracy: 0.7485
Epoch 3/10
50000/50000 [==============================] - 19s 371us/step - loss: 0.4859 - accuracy: 0.7546
Epoch 4/10
50000/50000 [==============================] - 19s 375us/step - loss: 0.4783 - accuracy: 0.7579
Epoch 5/10
50000/50000 [==============================] - 19s 372us/step - loss: 0.4657 - accuracy: 0.7653
Epoch 6/10
50000/50000 [==============================] - 19s 377us/step - loss: 0.4455 - accuracy: 0.7808
Epoch 7/10
50000/50000 [==============================] - 19s 377us/step - loss: 0.4372 - accuracy: 0.7867
Epoch 8/10
50000/50000 [==============================] - 18s 355us/step - loss: 0.4315 - accuracy: 0.7883
Epoch 9/10
50000/50000 [==============================] - 17s 344us/step - loss: 0.4298 - accuracy: 0.7904
Epoch 10/10
50000/5000

In [9]:
Z = cnn_model.classify(X_test)
img_patches_gt = create_patches_gt(y_test, 16, 16)
y_real = np.mean(img_patches_gt, axis=(1, 2)) > 0.25
print(get_classification_results(y_real, Z) )

0.80616


### SVM Model

In [12]:
from svm_model import SvmModel
svm_model = SvmModel()

initialize svm class


In [13]:
svm_model.initialize()
svm_model.train(y_train, X_train)

Create SVM Classifier with parameters, gamma= rbf, kernel=rbf, degree=4
Training Started
Training completed


In [14]:
Z = svm_model.classify(X_test)
    
# Calculate ground-truth labels
img_patches_gt = create_patches_gt(y_test, 16, 16)
y_real = np.mean(img_patches_gt, axis=(1, 2)) > 0.25
print(get_classification_results(y_real, Z)  )



0.71232
